# Regression Style Classification with Images

### Import Required Packages

In [ ]:
import pandas as pd
import numpy as np
!pip install pycaret
from pycaret.classification import *
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

### Read in the Data

In [ ]:
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(labels)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

### Display Data Examples

In [ ]:
y_train_df = pd.DataFrame(y_train)
y_train_df.reset_index(inplace = True, drop = False)
y_train_df.columns = ['index', 'label']

## Extract first example for each category
examples = y_train_df.groupby('label').first()
examples.reset_index(inplace = True, drop = False)

In [ ]:
#Display the example images in a grid
fig, ax = plt.subplots(2, 5)
for i in range(examples.shape[0]):
    if (i - 5) < 0:
        y = 0
    else:
        y = 1
    x = i % 5

    ax[y, x].imshow(x_train[examples['index'][i], :, :, :])
    ax[y, x].set_title(labels[i])
    ax[y, x].axis('off')
fig.tight_layout()

### Normalize and Reshape the Data

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], 32 * 32 * 3))
x_train = pd.DataFrame(x_train.astype('float32') / 255)
x_test = np.reshape(x_test, (x_test.shape[0], 32 * 32 * 3))
x_test = pd.DataFrame(x_test.astype('float32') / 255)

# Convert labels from integers to specific label text
y_train = pd.DataFrame(y_train)
y_train['label'] = ''
for i in range(y_train.shape[0]):
    y_train['label'][i] = labels[y_train.iloc[i,0]]
y_test = pd.DataFrame(y_test)
y_test['label'] = ''
for i in range(y_test.shape[0]):
    y_test['label'][i] = labels[y_test.iloc[i,0]]

# Add response column to predictor matrix
x_train['label'] = y_train['label']
x_test['label'] = y_test['label']

### Train Models with PyCaret
##### PyCaret is a package that allows for easyfitting of multiple models

In [ ]:
# Specify training parameters
s = setup(data=x_train, # training data
          target='label', # label column in training data
          test_data=x_test, # test data
          index=False,
          ordinal_features=None, # specify feature type if desired
          numeric_features=None, # specify feature type if desired
          categorical_features=None, # specify feature type if desired
          preprocess=True,
          max_encoding_ohe=25, # default. one-hot-encoding
          polynomial_features=False, # create x1^2, x1*x2, ...
          polynomial_degree=2,
          normalize=True,
          normalize_method='zscore',
          pca=False, # Principal component preprocessing
          pca_method='linear',
          use_gpu=True, # False if GPU is not available
          session_id=13)

In [ ]:
# View available models
models()

### The following cell is slow; it is fitting multiple models with repeated k-fold validation

In [ ]:
# Specify k-fold cross-validation and run the desired models from the above list
rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=13)

best = compare_models(include = ['lr', 'knn', 'nb', 'dt', 'svm', 'mlp', 'ridge', 'rf'],
                      fold=rkf,
                      cross_validation=False,
                      sort='Accuracy',
                      n_select=3)

In [ ]:
# Display confusion matrix for the best model
preds = predict_model(best[0], x_test)
CM_test = confusion_matrix(y_test['label'], preds['prediction_label'])
fig = sns.heatmap(CM_test, annot = True, fmt = 'g')
fig.set(xlabel='Prediction', ylabel='Truth', xticklabels=labels, title = 'Test Confusion Matrix')
fig.set_yticklabels(labels, va='center')
plt.show()

# Convolutional Neural Network Classification with Images

### Load Additional Packages

In [ ]:
from tensorflow.keras.utils import to_categorical

### Read in the Data, Normalize, Restructure

In [ ]:
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(labels)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Train a Basic CNN

In [ ]:
# Model Definition: 2 convolutional layers with max pooling followed by 2 dense layers with dropout
# The first to lines can be removed if not training on mulitple GPUs
mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
with mirrored_strategy.scope():
    parallel_model = tf.keras.models.Sequential()
    parallel_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (32, 32, 3)))
    parallel_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    parallel_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
    parallel_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    parallel_model.add(tf.keras.layers.Flatten())
    parallel_model.add(tf.keras.layers.Dense(128, activation = 'relu'))
    parallel_model.add(tf.keras.layers.Dropout(0.3))
    parallel_model.add(tf.keras.layers.Dense(len(labels), activation = 'softmax'))

In [ ]:
# Define the optimzer, loss, and metric to track during training
parallel_model.compile(optimizer = tf.keras.optimizers.Adamax(learning_rate = 0.0001),
                       loss = 'categorical_crossentropy',
                       metrics = ['acc'])

In [ ]:
# Train the model
history = parallel_model.fit(x_train, y_train, epochs=10)

In [ ]:
# Evaluate performance on test data
metrics = parallel_model.evaluate(x_test,  y_test, verbose=2)
loss, accuracy = metrics[0], metrics[1]

In [ ]:
# Generate confusion matrix on test data
preds = pd.DataFrame(parallel_model.predict(x_test))
preds = pd.DataFrame(preds.idxmax(axis=1))
preds.columns = ['index']
preds['label'] = preds['index'].apply(lambda x: labels[x])
y_test = pd.DataFrame(pd.DataFrame(y_test).idxmax(axis = 1))
y_test.columns = ['index']
y_test['label'] = y_test['index'].apply(lambda x: labels[x])


fig = sns.heatmap(confusion_matrix(y_true = y_test['label'], y_pred = preds['label']), annot = True, fmt = 'g')
fig.set(xlabel = 'Prediction', ylabel = 'Truth', xticklabels = labels, title = 'Test Confusion Matrix')
fig.set_yticklabels(labels, va = 'center')

In [ ]:
# Saliency mapping
img1 = x_train[examples['index'][0], :, :, :]
img = np.expand_dims(img1, axis=0)
y_pred = parallel_model.predict(img)
images = tf.Variable(img, dtype=float)

with tf.GradientTape() as tape:
    pred = parallel_model(images, training=False)
    class_idxs_sorted = np.argsort(pred.numpy().flatten())[::-1]
    loss = pred[0][class_idxs_sorted[0]]

grads = tape.gradient(loss, images)

dgrad_abs = tf.math.abs(grads)
dgrad_max_ = np.max(dgrad_abs, axis=3)[0]
# normalize to range between 0 and 1
arr_min, arr_max = np.min(dgrad_max_), np.max(dgrad_max_)
grad_eval = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)
grad_eval[grad_eval < np.percentile(0.4, grad_eval)] = np.nan

fig, ax = plt.subplots(1, 2, figsize=(8, 8))
ax[0].set_title('Original')
ax[0].imshow(img1)
ax[0].axis('off')
ax[1].set_title('Saliency Map')
ax[1].imshow(img1)
ax[1].imshow(grad_eval, cmap='Reds')
ax[1].axis('off')
plt.tight_layout()